In [77]:
#import support
import pandas as fu
import os
import glob
from datetime import datetime as dt
import json

Data for Subhub/MISP Keycloak data

type: str,
clientId: str,
userId: str,
ipAddress: str,
identity_provider_identity: str,
username: str,
loginDate: datetime

userId: str,
username: str,
org: str,
countSubhub: [ (day, #login), (day, #login)]




In [78]:
# sets the folder path to collect log data.
path1 = os.getcwd()
path2 = path1 + "\\Data\\"

#sets the raw imports.
data_list = glob.glob(os.path.join(path2, "*keycloak*.json"))

In [79]:
print('List of imported files:')
count = 0
for data in data_list:
    print(count, ': ', data.split('\\')[-1])
    count+=1

List of imported files:
0 :  (block) 20250118-20250512_keycloaklogs_114days.json
1 :  (block) 20250118-20250519_keycloaklogs_121days.json
2 :  20241213-20250313_keycloaklogs_30days_OPTION2.json
3 :  2024_12_18-2025-01_17_60days-90days_keycloaklogs.json
4 :  20250321-20250328_keycloaklogs_7days.json
5 :  20250328-20250404_keycloaklogs_7days.json
6 :  20250404-20250411_keycloaklogs_7days.json
7 :  20250411-20250418_keycloaklogs_full-log-groups_7days.json
8 :  20250418-20250425_keycloaklogs_7days.json
9 :  20250425-20250502_keycloaklogs_7days.json
10 :  2025_01_17-2025-02_16_30days-60days_keycloaklogs.json
11 :  2025_02_16-2025-03_18_0days-30days_keycloaklogs.json


In [80]:
# open files

content = []
for data in data_list:
    try:
        with open(data) as f:
            content.extend(json.load(f))
    except:
        #Prints which file failed to open, as well as gives you a location to go test why the json load possibly failed.
        print ("Failed: " + data)
        print ("Go to https://jsonlint.com/ to fix the error")

#Displays the total number of objects to be processed.
print("Total entries: " + str(len(content)))

Total entries: 111205


In [81]:
for entry in content:
    try:
        if '@message' in entry:
            pass
        else:
            print(entry)
    except:
        print('failed')


{'_section': 'MISP Logins'}
{'_section': 'OpenSearch Logins'}
{'_section': 'MISP Logins'}
{'_section': 'OpenSearch Logins'}


In [82]:
    
log_db = []
count = 0
tempDate = str
tempTime = str
logKeys = ['type', 'clientId', 'userId', 'username', 'identity_provider_identity']

idstore = []


for entry in content:

    tempLog = {}

    try:

        if '@message' in entry:

            tempLog['id'] = entry['@message']['json']['time']

            logitem = entry['@message']['json']['log'].split(',')

            tempDate, tempTime = entry['@message']['json']['time'].split('T')

            tempLog['year'] = int(tempDate.split('-')[0]) 
            tempLog['month'] = int(tempDate.split('-')[1])
            tempLog['day'] = int(tempDate.split('-')[2])        
            tempLog['time'] = tempTime.split('Z')[0]

            for item in logitem:
                if '=' not in item:
                    pass
                else:
                    item = item.split('=')
                    item[0] = item[0].split(' ')[-1]
                    if item[0] in logKeys:
                        tempLog[item[0]]= item[1]
            if 'type' in tempLog.keys():
                if tempLog['id'] not in idstore:
                    idstore.append(tempLog['id'])
                    log_db.append(tempLog)
    except:
       # print(str(count) + '. Failed entry: ' + entry['@message'])
       pass
    count += 1


In [83]:
for item in log_db:
    #print(item)
    break

In [84]:
for log in log_db:
    for key in log:
        if key == 'clientId':
            if 'https' in log[key]:
                log[key] = 'metadata'
                #print(log[key])

In [85]:
userdataloc = glob.glob(os.path.join(path2, "response*.json"))
userList = []


try:
    with open(userdataloc[0], "r") as f:
        userList = json.load(f)
        #for line in f:
         #   line = line.strip('\n')
          #  if count == 0:
           #     userKey = line.split(',')
            #    count += 1
            #else:
             #   userList.append(dict(zip(userKey, line.split(','))))
                #print(userList[count-1])
              #  count+=1
            #userList.append(line)
            #print(line)
except:
    print('failed')

#print(userList)

In [86]:
userdataloc = glob.glob(os.path.join(path2, "*.csv"))


In [87]:
userOrgList = []

try:
    count = 0
    with open(userdataloc[0], 'r') as f:
        for line in f:
            line = line.strip('\n')
            line = line.rstrip('"')
            line = line.lstrip('"')
            #print(line)
            if count == 0:
                line = line.split('","')
                userKey = line
                #print(userKey)
                count += 1
            else:
                line = line.split('","')
                userOrgList.append(dict(zip(userKey, line)))
                #print(userOrgList[count-1])
                count+=1
        
except:
    pass

#print(userOrgList)

In [88]:
#print(userList[0])
userList2 = []
for n in userList['data']['Users']:
    #print(n)
    userList2.append(n)

#print(userList2)    
#for u in userList2:
    #print(u['id'] + ', ' + u['email'])
    

In [89]:
missingUser = ""
for user in userList2:
    orgFlag = False
    for org in userOrgList:
        if user['email'] == org['Email']:
            user['org'] = org['RK-Organization']
            txt = user['username']
            #print(f'Username: {txt}')
            orgFlag = True
    if orgFlag == False:
        #print(user['username'])
        txt = user['username']
        missingUser = missingUser + txt + ', '

missingUser = missingUser.rstrip(', ')
missingUser = missingUser.split(', ')

In [90]:
#Used to track the users that are in the RK system user list but not the RKR Jira user list.  This will identify potentially the developers if done correctly.
count = 1

for user in missingUser:
    #print(str(count)+ ': ' + user)
    count+=1

#print(len(missingUser.split(',')))


In [91]:
missingOrg = ''
txt = ''
for org in userOrgList:
    userFlag = False
    for user in userList2:
        if org['Email'] == user['email']:
            userFlag = True
            #print 
            pass
    if userFlag == False:
        txt = org['Email']
        missingOrg = missingOrg + txt + ', '

missingOrg = missingOrg.rstrip(', ')
missingOrg = missingOrg.split(', ')



In [92]:
#used to track the users that are in the RKR jira list but not in the RK system user list
count = 1
for user in missingOrg:
    #print(str(count) +': '+user)
    count+=1

#print(len(missingOrg.split(',')))

In [93]:
UserInfo = []
tempUser = {}

#UserInfo = [{'userId': 'f670759e-2af0-4114-977b-b3be9182e882', 'username': 'johnathon.murray.ctr'}]

for log in log_db:
    if 'userId' and 'username' in log.keys():
        tempUser = {'userId': log['userId'], 'username': log['username']}
    if tempUser in UserInfo:
        pass
    else:
        #print(tempUser)
        UserInfo.append(tempUser)

try:
    UserInfo.remove({})
except:
    pass

print(len(UserInfo))

112


In [95]:

#for user in UserInfo:
    #print(user['username'])

#print(len(UserInfo))

In [96]:
count = 0
for log in log_db:
    if 'username' not in log.keys():
        for user in UserInfo:
            if user['userId'] == log['userId']:
                log_db[count]['username'] = user['username']
                
    count+=1

In [97]:
count = 0
for log in log_db:
    for user in userList2:
        if user["id"] == log['userId']:
            #print(user['Gov Organization'])
            log_db[count]['RK-Org'] = user['attributes']['gov-organization']
            if user['username'] not in missingUser:
                #sets the org to what has been defined as the users owning org per the Request for acount ticket.
                log_db[count]['org'] = user['org']
            else:
                #sets org to vendor that is developing/maintaining the system.  Backend user accounts.
                log_db[count]['org'] = 'BAH' 
    count +=1


In [98]:
#I don't remember the purpose of this code, will need to research.

columnsList = []
count = 0

for item in log_db:
    #print(item.keys())
    #if 'type' in item.keys():
        #print(item['type'])
    for k in item.keys():
        if k in columnsList:
            pass
        else:
            columnsList.append(k)

In [99]:
df_loginfo = fu.DataFrame(log_db)

#print(df_loginfo)

In [100]:

file_output = dt.now()

file_output = file_output.strftime('%Y%m%d') + '_UserInfo.xlsx'

print(file_output)


20250523_UserInfo.xlsx


In [76]:
df_loginfo.to_excel(file_output, sheet_name= 'logs', index=False)